In [1]:
import pandas as pd
import statistics

In [2]:
df = pd.read_csv('../datasets/processed_dataset.csv')

In [3]:
numerical_grades = {'A+': 4.1, 'A': 4.0, 'A-': 3.7, 'B+': 3.3, 'B': 3.0, 'B-': 2.7, 'C+': 2.3, 'C': 2.0,
                    'C-': 1.7, 'D+': 1.3, 'D': 1.0, 'D-': 0.5, 'F': 0.0}

In [4]:
subject = [course_code.split()[0] for course_code in df['Course Code']]

In [5]:
df['Subject'] = subject

In below, we extract the letter grade rates which will be used later in clustering. The rates are calculated cumulatively right from the first semester.

In [6]:
def get_grade_rate(df):
    rate_dictionary = {}
    for row_idx in df.index:
        semester = df.iloc[row_idx, 14]
        rate_dictionary.setdefault(semester, {})
        course_code = df.iloc[row_idx, 0]
        rate_dictionary[semester].setdefault(course_code, {})        
        letter_grade = df.iloc[row_idx, 5]
        
        # retrieving previous letter grade counts from the time series
        current_count = 0
        current_sem_idx = sorted(rate_dictionary).index(semester)
        for i in range(current_sem_idx-1, -1, -1):
            prev_semester_name = sorted(rate_dictionary)[i]
            if course_code in rate_dictionary[prev_semester_name]:
                if letter_grade in rate_dictionary[prev_semester_name][course_code]:
                    current_count = rate_dictionary[prev_semester_name][course_code][letter_grade]
                    break
        
        if letter_grade not in rate_dictionary[semester][course_code]:
            rate_dictionary[semester][course_code][letter_grade] = current_count + 1
        else:
            rate_dictionary[semester][course_code].setdefault(letter_grade, 0)
            rate_dictionary[semester][course_code][letter_grade] += 1
    
    # calculating rates
    for semester in sorted(rate_dictionary):
        for course_code in rate_dictionary[semester]:
            sum_vals = sum(rate_dictionary[semester][course_code].values())
            for letter_grade in rate_dictionary[semester][course_code]:
                rate_dictionary[semester][course_code][letter_grade] /= sum_vals
        
    return rate_dictionary

In [7]:
rates = get_grade_rate(df)

In [8]:
grade_rates = {}
for row_idx in df.index:
    semester = df.iloc[row_idx, 14]
    course_code = df.iloc[row_idx, 0]
    
    for letter_grade in numerical_grades:
        grade_rates.setdefault(letter_grade, [])
        if letter_grade in rates[semester][course_code]:
            grade_rates[letter_grade].append(rates[semester][course_code][letter_grade])
        else:
            grade_rates[letter_grade].append(0)

for letter_grade in grade_rates:
    df[letter_grade + ' rate'] = grade_rates[letter_grade]

In below, we extract features namely, mean of students GPA taking the course and standard deviation of students GPA taking the course.

In [9]:
def get_gpa_stats(df):
    course_stats = {}
    
    for row_idx in df.index:
        semester = df.iloc[row_idx, 14]
        course_code = df.iloc[row_idx, 0]
        gpa = df.iloc[row_idx, 7]
        letter_grade = df.iloc[row_idx, 5]
        grade = numerical_grades[letter_grade]
        course_stats.setdefault(semester, {})
        course_stats[semester].setdefault(course_code, {})
        course_stats[semester][course_code].setdefault('gpa', [])
        course_stats[semester][course_code].setdefault('grade', [])
        
        current_gpas = []
        current_grades = []
        current_sem_idx = sorted(course_stats).index(semester)
        for i in range(current_sem_idx-1, -1, -1):
            prev_semester_name = sorted(course_stats)[i]
            if course_code in course_stats[prev_semester_name]:
                current_gpas = course_stats[prev_semester_name][course_code]['gpa']
                current_grades = course_stats[prev_semester_name][course_code]['grade']
                break        
        
        if course_stats[semester][course_code]['gpa'] == []:
            course_stats[semester][course_code]['gpa'] += current_gpas + [gpa]
            course_stats[semester][course_code]['grade'] += current_grades + [grade]
        else:
            course_stats[semester][course_code]['gpa'].append(gpa)
            course_stats[semester][course_code]['grade'].append(grade)
            
    return course_stats

In [10]:
stats = get_gpa_stats(df)

In [11]:
mean_gpa, mean_grade = [], []
std_dev_gpa, std_dev_grade = [], []
for row_idx in df.index:
    semester = df.iloc[row_idx, 14]
    course_code = df.iloc[row_idx, 0]
    
    gpas = stats[semester][course_code]['gpa']
    grades = stats[semester][course_code]['grade']
    
    mean_gpa.append(statistics.mean(gpas))
    mean_grade.append(statistics.mean(grades))
    
    if len(gpas) < 2:   # assuming the gpa/grade itself rather than it's stdev, because at minimum 2 data -
                        # points is required for calculating the variance.
        std_dev_gpa.append(gpas[0])
        std_dev_grade.append(grades[0])
    else:
        std_dev_gpa.append(statistics.stdev(gpas))
        std_dev_grade.append(statistics.stdev(grades))

df['Mean GPA - Students taken'] = mean_gpa   # mean of students GPA who have taken the course so far
df['Mean Grade - Students taken'] = mean_grade   # mean of course grades from students who have taken it
df['STDEV GPA - Students taken'] = std_dev_gpa   # std. dev. of students GPA who have taken the course so far
df['STDEV Grade - Students taken'] = std_dev_grade   # std. dev. of course grades from students who have taken it

In [12]:
df.head()

,Course Code,Course Title,Student Number,Department Code,Course Level,Letter Grade,Status,GPA,Standing,Completed Credits,...,C rate,C- rate,D+ rate,D rate,D- rate,F rate,Mean GPA - Students taken,Mean Grade - Students taken,STDEV GPA - Students taken,STDEV Grade - Students taken
0,UNI 111,Critical Reading & Writing in Turkish I,240,SOC,Undergraduate,F,Unsuccessful,2.62,Freshman,18,...,0.054422,0.040816,0.047619,0.020408,0.020408,0.088435,2.466190,2.680952,0.865985,1.193667
1,UNI 107,World Civilizations& Global Encounters I,338,PSY,Undergraduate,A,Successful,3.68,Freshman,18,...,0.090909,0.136364,0.045455,0.000000,0.045455,0.000000,2.893182,2.986364,0.821569,1.098612
2,UNI 105,Understanding Society and Culture I,338,PSY,Undergraduate,A,Successful,3.68,Freshman,18,...,0.038462,0.000000,0.000000,0.000000,0.000000,0.038462,2.713846,3.211538,0.952080,0.937369
3,UNI 203,Understanding Science and Technology,338,PSY,Undergraduate,A,Successful,3.68,Freshman,18,...,0.081967,0.032787,0.000000,0.021858,0.027322,0.065574,2.479399,2.813115,0.980915,1.117219
4,UNI 105,Understanding Society and Culture I,240,SOC,Undergraduate,A,Successful,2.62,Freshman,18,...,0.038462,0.000000,0.000000,0.000000,0.000000,0.038462,2.713846,3.211538,0.952080,0.937369


In [13]:
df.to_csv('processed_course_clustering_dataset.csv', index=False)